In [1]:
import numpy as np

In [2]:
def GetF2(G,r):
    n = r.shape[0]
    v = np.zeros_like(r)
    for i in range(n):
        v[i] = G[i](r[0],r[1])
    return v

In [3]:
def GetF3(G,r):
    n = r.shape[0]
    v = np.zeros_like(r)
    for i in range(n):
        v[i] = G[i](r[0],r[1],r[2])
    return v

In [4]:
def GetJacobian2(f,r,h=1e-6):
    n = r.shape[0]
    J = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            rf = r.copy()
            rb = r.copy()
            rf[j] = rf[j] + h
            rb[j] = rb[j] - h
            J[i,j] = ( f[i](rf[0],rf[1]) - f[i](rb[0],rb[1])  )/(2*h)
    return J

In [5]:
def GetJacobian3(f,r,h=1e-6):
    n = r.shape[0]
    J = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            rf = r.copy()
            rb = r.copy()
            rf[j] = rf[j] + h
            rb[j] = rb[j] - h
            J[i,j] = ( f[i](rf[0],rf[1],rf[2]) - f[i](rb[0],rb[1],rb[2])  )/(2*h)
    return J

In [6]:
def NewtonRaphson2(G,r,itmax=1000,error=1e-9):
    it = 0
    d = 1.
    dvector = []
    while d > error and it < itmax:
        # Vector actual
        rc = r
        F = GetF2(G,rc)
        J = GetJacobian2(G,rc)
        InvJ = np.linalg.inv(J)
        r = rc - np.dot(InvJ,F)
        diff = r - rc
        d = np.max( np.abs(diff) )
        dvector.append(d)
        it += 1
    return r,dvector

In [7]:
def NewtonRaphson3(G,r,itmax=1000,error=1e-9):
    it = 0
    d = 1.
    dvector = []
    while d > error and it < itmax:
        # Vector actual
        rc = r
        F = GetF3(G,rc)
        J = GetJacobian3(G,rc)
        InvJ = np.linalg.inv(J)
        r = rc - np.dot(InvJ,F)
        diff = r - rc
        d = np.max( np.abs(diff) )
        dvector.append(d)
        it += 1
    return r,dvector

In [8]:
def Metric2(G,r):
    return 0.5*np.linalg.norm(GetF2(G,r))**2

In [9]:
def Metric3(G,r):
    return 0.5*np.linalg.norm(GetF3(G,r))**2

In [10]:
def GetFig(R,M,it):
    fig = plt.figure(figsize=(6,3))
    ax = fig.add_subplot(1,2,1)
    ax1 = fig.add_subplot(1,2,2)
    ax.plot(R[:it])
    ax1.plot(M[:it])
    plt.show()

In [11]:
def Minimizer2(G,r,lr=1e-2,epochs=int(1e4),error=1e-7):
    metric = 1
    it = 0
    M = np.array([])
    R = np.array([r])
    while metric > error and it < epochs:
        M = np.append(M,Metric2(G,r))
        J = GetJacobian2(G,r)
        Vector = GetF2(G,r)
        # Machine learning
        r -= lr*np.dot(J,Vector)
        R = np.vstack((R,r))
        metric = Metric2(G,r)
        it += 1
        """if it%50 == 0:
            clear_output(wait=True)
            GetFig(R,M,it)
            time.sleep(0.001)"""
    return r

In [12]:
def Minimizer3(G,r,lr=1e-2,epochs=int(1e4),error=1e-7):
    metric = 1
    it = 0
    M = np.array([])
    R = np.array([r])
    while metric > error and it < epochs:
        M = np.append(M,Metric3(G,r))
        J = GetJacobian3(G,r)
        Vector = GetF3(G,r)
        # Machine learning
        r -= lr*np.dot(J,Vector)
        R = np.vstack((R,r))
        metric = Metric3(G,r)
        it += 1
        """if it%50 == 0:
            clear_output(wait=True)
            GetFig(R,M,it)
            time.sleep(0.001)"""
    return r

In [13]:
G = np.array([lambda x, y: np.log(x**2 + y**2)-np.sin(x*y)-np.log(2)-np.log(np.pi), 
              lambda x, y: np.exp(x-y) + np.cos(x*y)])

In [14]:
r, dvector = NewtonRaphson2(G, np.array([2., 2.]))
r

array([1.77245385, 1.77245385])

In [15]:
xsol = Minimizer2(G, np.array([2., 2.]))
xsol

array([1.77231221, 1.77269214])

In [16]:
G2 = np.array([lambda x,y,z: 6*x - 2*np.cos(y*z) - 1.,
               lambda x,y,z: 9*y + np.sqrt(x**2 + np.sin(z) + 1.06) + 0.9,
               lambda x,y,z: 60*z + 3*np.exp(-(x*y)) + 10*np.pi - 3.])

In [17]:
r, dvector = NewtonRaphson3(G2, np.array([0., 0., 0.]))
r

array([ 0.49814468, -0.1996059 , -0.52882598])

In [18]:
xsol = Minimizer3(G2, np.array([0., 0., 0.]))
xsol

C:\Users\brune\AppData\Local\Temp\ipykernel_6660\1689576832.py:3: RuntimeWarning: overflow encountered in exp
  lambda x,y,z: 60*z + 3*np.exp(-(x*y)) + 10*np.pi - 3.])
C:\Users\brune\AppData\Local\Temp\ipykernel_6660\2793432053.py:10: RuntimeWarning: invalid value encountered in scalar subtract
  J[i,j] = ( f[i](rf[0],rf[1],rf[2]) - f[i](rb[0],rb[1],rb[2])  )/(2*h)


array([ inf, -inf,  nan])